In [ ]:
# ==========================================
# SETUP BLOCK 
# ==========================================

import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ---- Imports from project files ----
sys.path.append(os.path.abspath(".."))
from Helper_funtions import (
    clean_up_subjects,
    calculate_true_false_score,
    calculate_Internet_terms_understanding_score,
    group_internet_understanding,
    analyze_distribution,
    analyze_subject_distribution
)
from lists import (
    demographic_columns,
    multiple_choice_questions,
    single_choice_questions
)
from answer_categories import question_orders, LABEL_MAPS

# ---- General plot style ----
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)

# ---- Data loading ----
DATA_FILE = os.path.join("..", "Data", "Fertige Tabelle.xlsx")
df = pd.read_excel(DATA_FILE)
df.columns = df.columns.str.strip()

# Clean up multi-subject columns
for col in ["Most used subjects", "Preferred Subjects", "Least preferred Subjects"]:
    if col in df.columns:
        df = clean_up_subjects(df, col)

# Calculate additional scores
if all(q in df.columns for q in ["True/False_1", "True/False_2"]):
    df = calculate_true_false_score(df)

if any(col.startswith("Internet terms_") for col in df.columns):
    df = calculate_Internet_terms_understanding_score(df)
    df = group_internet_understanding(df)

print("✅ Setup complete – DataFrame loaded and preprocessed")
print(f"Rows: {len(df)}, Columns: {len(df.columns)}")


In [ ]:
# --------- Shows distribution in table form -----------

# Filter: only non-demographic questions that actually exist in df
non_demo_single = [c for c in single_choice_questions if c not in demographic_columns and c in df.columns]
non_demo_multi  = [c for c in multiple_choice_questions if c not in demographic_columns and c in df.columns]

# ---- Single-Choice Questions ----
for column in non_demo_single:
    analyze_distribution(df, column, f"🔹 {column}")

# ---- Multiple-Choice Questions ----
for column in non_demo_multi:
    analyze_subject_distribution(df, column)



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# USEFULNESS --------------------------

# Deine Tabelle (kann auch direkt aus DataFrame kommen)
data = {
    "Usefullness AI": ["Sehr nützlich", "Ziemlich nützlich", "Teils/teils", "Wenig nützlich"],
    "Count": [65, 73, 51, 4],
    "Percentage": [33.7, 37.8, 26.4, 2.1]
}
df_usefulness = pd.DataFrame(data).set_index("Usefullness AI")

# Übersetzungen
label_map = {
    "Sehr nützlich": "Very useful",
    "Ziemlich nützlich": "Quite useful",
    "Teils/teils": "Neutral",
    "Wenig nützlich": "Slightly useful"
}

# Labels übersetzen
df_usefulness.index = df_usefulness.index.map(label_map)

# Plot
ax = df_usefulness["Percentage"].plot(
    kind="bar",
    color="#1f77b4", edgecolor="black"
)

ax.set_title("Usefulness of AI tools", fontsize=16, fontweight="bold")
ax.set_ylabel("Percentage of respondents (%)", fontsize=14, fontweight="bold")
ax.set_xlabel("", fontsize=14, fontweight="bold")

ax.tick_params(axis="x", rotation=45, labelsize=12, width=2)
ax.tick_params(axis="y", labelsize=12, width=2)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# TEACHERS LESSONS -------------------

# Tabelle direkt in DataFrame
data = {
    "Teachers preparing lessons": [
        "Finde ich sehr gut",
        "Finde ich gut",
        "Neutral / Mir egal",
        "Stört mich ein wenig",
        "Stört mich sehr"
    ],
    "Count": [4, 14, 115,48, 22 ],
    "Percentage": [2.0, 6.9,56.7,23.6, 10.8]
}
df_teachers = pd.DataFrame(data).set_index("Teachers preparing lessons")

# Übersetzungen
label_map = {
    "Stört mich sehr": "Bothers me a lot",
    "Stört mich ein wenig": "Bothers me a bit",
    "Neutral / Mir egal": "Neutral",
    "Finde ich gut": "I like it",
    "Finde ich sehr gut": "I really like it"
}

# Labels übersetzen
df_teachers.index = df_teachers.index.map(label_map)

# Plot
ax = df_teachers["Percentage"].plot(
    kind="bar",
    color="#1f77b4", edgecolor="black"
)

ax.set_title("Teachers preparing lessons", fontsize=16, fontweight="bold")
ax.set_ylabel("Percentage of respondents (%)", fontsize=14, fontweight="bold")
ax.set_xlabel("", fontsize=14, fontweight="bold")

ax.tick_params(axis="x", rotation=45, labelsize=12, width=2)
ax.tick_params(axis="y", labelsize=12, width=2)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# TEACHERS GRADES -----------------

# Tabelle in DataFrame
data = {
    "Teachers giving grades": [
        "Finde ich sehr gut",
        "Finde ich gut",
        "Neutral / Mir egal",
        "Stört mich ein wenig",
        "Stört mich sehr"
    ],
    "Count": [3, 7, 34, 51, 108],
    "Percentage": [ 1.5, 3.4, 16.7, 25.1, 53.2]
}
df_grades = pd.DataFrame(data).set_index("Teachers giving grades")

# Übersetzungen
label_map = {
    "Stört mich sehr": "Bothers me a lot",
    "Stört mich ein wenig": "Bothers me a bit",
    "Neutral / Mir egal": "Neutral",
    "Finde ich gut": "I like it",
    "Finde ich sehr gut": "I really like it"
}

# Labels übersetzen
df_grades.index = df_grades.index.map(label_map)

# Plot
ax = df_grades["Percentage"].plot(
    kind="bar",
    color="#1f77b4", edgecolor="black"
)

ax.set_title("Teachers giving grades", fontsize=16, fontweight="bold")
ax.set_ylabel("Percentage of respondents (%)", fontsize=14, fontweight="bold")
ax.set_xlabel("", fontsize=14, fontweight="bold")

ax.tick_params(axis="x", rotation=45, labelsize=12, width=2)
ax.tick_params(axis="y", labelsize=12, width=2)

plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# PUPROSES -----------------

# Tabelle
data = {
    "Purposes to use AI": [
        "Erklärungen",
        "Zusammenfassungen",
        "Grammatik / Rechtschreibung",
        "Schreiben von Texten",
        "Ideenfindung / Kreative Ideen",
        "Vorbereiten von Vorträgen",
        "Lerntechniken / Tipps wie man lernt"
    ],
    "Mentions": [153, 119, 90, 72, 68, 63, 36],
    "Percentage of respondents": [84.1, 65.4, 49.5, 39.6, 37.4, 34.6, 19.8]
}
df_purposes = pd.DataFrame(data).set_index("Purposes to use AI")

# Übersetzungen (aus Screenshot)
label_map = {
    "Erklärungen": "Explanations",
    "Zusammenfassungen": "Summaries",
    "Grammatik / Rechtschreibung": "Grammar / Spelling",
    "Schreiben von Texten": "Writing texts",
    "Ideenfindung / Kreative Ideen": "Brainstorming / Creative ideas",
    "Vorbereiten von Vorträgen": "Preparation of lectures",
    "Lerntechniken / Tipps wie man lernt": "Learning techniques / tips how to learn"
}

# Labels übersetzen
df_purposes.index = df_purposes.index.map(label_map)

# Plot
ax = df_purposes["Percentage of respondents"].plot(
    kind="bar",
    color="#1f77b4", edgecolor="black"
)

ax.set_title("Purposes to use AI", fontsize=16, fontweight="bold")
ax.set_ylabel("Percentage of respondents (%)", fontsize=14, fontweight="bold")
ax.set_xlabel("", fontsize=14, fontweight="bold")

ax.tick_params(axis="x", rotation=45, labelsize=12, width=2)
ax.tick_params(axis="y", labelsize=12, width=2)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# MOTIVATIONS ---------------

# Tabelle
data = {
    "Reasons to use AI": [
        "Zeitersparnis",
        "Als Prüfungsvorbereitung",
        "Bessere Erklärungen als von den Lehrern",
        "weniger Stress",
        "Für Fragen die ich nicht im Klassenraum stellen..."
    ],
    "Percentage of respondents": [72.9, 57.1, 54.2, 32.8, 32.2]
}
df_motivations = pd.DataFrame(data).set_index("Reasons to use AI")

# Übersetzungen (aus Screenshot)
label_map = {
    "Zeitersparnis": "Saving time",
    "Als Prüfungsvorbereitung": "Preparation for exams",
    "Bessere Erklärungen als von den Lehrern": "Better explanations than from teachers",
    "weniger Stress": "Less stress",
    "Für Fragen die ich nicht im Klassenraum stellen...": "Questions I didn’t want to ask in class"
}

# Labels übersetzen
df_motivations.index = df_motivations.index.map(label_map)

# Plot
ax = df_motivations["Percentage of respondents"].plot(
    kind="bar",
    color="#1f77b4", edgecolor="black"
)

ax.set_title("Motivations to use AI", fontsize=16, fontweight="bold")
ax.set_ylabel("Percentage of respondents (%)", fontsize=14, fontweight="bold")
ax.set_xlabel("", fontsize=14, fontweight="bold")

ax.tick_params(axis="x", rotation=45, labelsize=12, width=2)
ax.tick_params(axis="y", labelsize=12, width=2)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# RELIABILITY ---------------

# Tabelle
data = {
    "Reliability AI": [
        "Sehr verlässlich",
        "Eher verlässlich",
        "Teils/teils",
        "Wenig verlässlich",
        "Gar nicht verlässlich",
        "Unsicher / Ich habe keine Meinung"
    ],
    "Count": [10, 84, 86, 15, 3, 5],
    "Percentage": [4.9, 41.4, 42.4, 7.4, 1.5, 2.5]
}
df_reliability = pd.DataFrame(data).set_index("Reliability AI")

# Übersetzungen
label_map = {
    "Sehr verlässlich": "Really reliable",
    "Eher verlässlich": "Rather reliable",
    "Teils/teils": "Neither reliable nor unreliable",
    "Wenig verlässlich": "Little reliable",
    "Gar nicht verlässlich": "Not reliable at all",
    "Unsicher / Ich habe keine Meinung": "Unsure / I don't have an opinion"
}
df_reliability.index = df_reliability.index.map(label_map)

# Plot
fig, ax = plt.subplots(figsize=(8, 5))

# Hauptkategorien
df_reliability["Percentage"].plot(
    kind="bar",
    color="#1f77b4", edgecolor="black",
    ax=ax, width=0.8, position=0
)

# Achsen/Titel
ax.set_title("Reliability of AI", fontsize=16, fontweight="bold")
ax.set_ylabel("Percentage of respondents (%)", fontsize=14, fontweight="bold")
ax.set_xlabel("", fontsize=14, fontweight="bold")

ax.tick_params(axis="x", rotation=45, labelsize=12, width=2)
ax.tick_params(axis="y", labelsize=12, width=2)

plt.tight_layout()
plt.show()




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Tabelle
data = {
    "Use AI school and freetime": [
        "Täglich",
        "Mehrmals pro Woche",
        "Etwa 1 Mal pro Woche",
        "Seltener",
        "Nie"
    ],
    "Count": [56, 68, 38, 31, 10],
    "Percentage": [27.6, 33.5, 18.7, 15.3, 4.9]
}
df_ai_use = pd.DataFrame(data).set_index("Use AI school and freetime")

# Übersetzungen aus Screenshot
label_map = {
    "Täglich": "Daily",
    "Mehrmals pro Woche": "Several times per week",
    "Etwa 1 Mal pro Woche": "About once per week",
    "Seltener": "Rarely",
    "Nie": "Never"
}
df_ai_use.index = df_ai_use.index.map(label_map)

# Plot
ax = df_ai_use["Percentage"].plot(
    kind="bar",
    color="#1f77b4", edgecolor="black"
)

ax.set_title("Use of AI tools for school and free time", fontsize=16, fontweight="bold")
ax.set_ylabel("Percentage of respondents (%)", fontsize=14, fontweight="bold")
ax.set_xlabel("", fontsize=14, fontweight="bold")

ax.tick_params(axis="x", rotation=45, labelsize=12, width=2)
ax.tick_params(axis="y", labelsize=12, width=2)

plt.tight_layout()
plt.show()
